## 1. PCA

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../00. 데이터 탐색 및 전처리/last_df.csv', index_col= 0)
# df.info()

X = df.iloc[:, :-2]
target_1 = df.iloc[:, -2:-1]
target_2 = df[['L.Vac']]

X.info()
target_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8143 entries, 2016-01-01 07:00:00 to 2016-12-31 23:00:00
Data columns (total 94 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   X1WOS45106_PV                          8143 non-null   float64
 1   X1WOS45109_PV                          8143 non-null   float64
 2   DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV     8143 non-null   float64
 3   N2.TO.EA1313A.1FI33503A_PV             8143 non-null   float64
 4   ROOF.HEATING.Zone.1.1JIC31101.PV       8143 non-null   float64
 5   ROOF.HEATING.Zone.2.1JIC31102.PV       8143 non-null   float64
 6   ROOF.HEATING.Zone.3.1JIC31103.PV       8143 non-null   float64
 7   ROOF.HEATING.Zone.4.1JIC31104.PV       8143 non-null   float64
 8   ATM.FLOW.SECT.9.N2.1FIC33415.PV        8143 non-null   float64
 9   Gross.width                            8143 non-null   float64
 10  UPPER.AB1.Z1.TMP.1TIC41101.PV          8143 

In [2]:
from sklearn.decomposition import PCA

pca = PCA(n_components= 0.95, random_state= 42)
X_pca = pca.fit_transform(X)
pca.n_components_ # 42개의 피쳐

# 생각해보니 pca는 새로운 피쳐로 피쳐 추출을 하는 방법이라 원래의 공정 중 중요한 공정을 알아볼 방법이 없네

42

## 2. KNN

In [3]:
# X.info() # 94 columns

# 머신러닝 10개의 피쳐를 가진 예측 문제에서 선형 회귀 모델을 학습하려면 100~500개의 데이터 필요
# 딥러닝 10개의 클래스를 분류하는 CNN모델의 경우 클래스당 1000개 이상 필요

# 현재 약 100개의 피쳐

# 해결 방법
# 1) 언더샘플링 - 무작위 샘플링하여 데이터 비율 맞추는 방법(데이터 손실 주의)
# 2) 오버샘플링 - 소수 클래스를 중복 생성하여 비율 맞추는 방법(과적합 주의)
# 3) 차원 축소
# 4) 클래스 가중치 조정 - 소수 클래스에 더 큰 가중치 부여 : sklearn의 class_weight 매개변수
# 5) 앙상블 모델 - 여러 모델을 조합하여 성능을 내는 방법 - 랜덤 포레스트나 배깅과 같은 앙상블 기법

# 평가 지표 선택
# 불균형 데이터의 경우 정확도만으로 성능 평가가 어려움
# 정밀도, 재현율, F1 점수, AUC-ROC

# 랜덤 포레스트/XGBoost와 같은 머신 러닝 모델 시도
# 충분한 성능을 얻지 못한다면 데이터 증강 및 오버 샘플링을 사용하여 딥러닝 모델 시도

In [7]:
target_1.info()

# 상위 10%
target1_threshold = np.percentile(target_1['L.RD'], 95)
target_1['target1_defect'] = np.where(target_1['L.RD'] > target1_threshold, 1, 0) # 불량이 아닌 경우 0, 불량인 경우 1
target_1['target1_defect'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 8143 entries, 2016-01-01 07:00:00 to 2016-12-31 23:00:00
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   L.RD            8143 non-null   float64
 1   target1_defect  8143 non-null   int32  
dtypes: float64(1), int32(1)
memory usage: 159.0+ KB


C:\Users\chjin\AppData\Local\Temp\ipykernel_12524\1417299523.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_1['target1_defect'] = np.where(target_1['L.RD'] > target1_threshold, 1, 0) # 불량이 아닌 경우 0, 불량인 경우 1


target1_defect
0    7741
1     402
Name: count, dtype: int64

In [ ]:
# 아니면 사분위수 이용해서 이상치 설정하자
# 폐기율 자체가 > 0.01이면 불량으로 본다


In [5]:
import numpy as np
from collections import Counter

# 데이터 분리
X_minority = X[y == 1]  # 소수 클래스 데이터
X_majority = X[y == 0]  # 다수 클래스 데이터

# 소수 클래스 샘플 복제
n_majority = len(X_majority)
n_minority = len(X_minority)
n_to_generate = n_majority - n_minority  # 생성할 샘플 수

# 복제 샘플 생성
indices = np.random.choice(len(X_minority), size=n_to_generate, replace=True)
X_oversampled = np.vstack([X, X_minority[indices]])
y_oversampled = np.hstack([y, np.ones(n_to_generate)])

print(f"After Oversampling: {Counter(y_oversampled)}")

NameError: name 'y' is not defined

In [ ]:
target_2.info()

# 상위 10%
target1_threshold = np.percentile(target_2['L.Vac'], 95)
target_2['target2_defect'] = np.where(target_2['L.Vac'] > target1_threshold, 1, 0)
target_2['target2_defect'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 8143 entries, 2016-01-01 07:00:00 to 2016-12-31 23:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   L.Vac   8143 non-null   float64
dtypes: float64(1)
memory usage: 127.2+ KB


C:\Users\chjin\AppData\Local\Temp\ipykernel_11220\906974176.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_2['target2_defect'] = np.where(target_2['L.Vac'] > target1_threshold, 1, 0)


target2_defect
0    7338
1     805
Name: count, dtype: int64

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = [10, 30, 50, 70, 90]

for n in n_neighbors :
  knn_clf = KNeighborsClassifier(n_neighbors= n)
  knn_clf.